# Preparation and Configuration

Python imports

In [ ]:
# dealing with datasets in Python
import pandas as pd
import geopandas as gpd

# db connectivity
import sqlalchemy

# load osm data
import osmnx as ox

# visualization of spatial data
from keplergl import KeplerGl
from shapely import wkt

Filter kepler.gl (i.e. "all") warnings until I know better

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Database connection details

In [ ]:
hdb_host = 'xxx.xxx.xxx.xxx'
hdb_port = xxxxx
hdb_user = 'xxx'
hdb_password = 'xxx'

hdb_schema = 'xxx'

connection_string = 'hana://%s:%s@%s:%s' % (hdb_user, hdb_password, hdb_host, hdb_port)

Enable inline SQL for readability

In [ ]:
%reload_ext sql
%config SqlMagic.displaylimit = 100
%sql $connection_string
%sql SET SCHEMA $hdb_schema

Configure the SRS that is going to be used. Choose one which is suitable for Porto.

In [ ]:
srid = 5018

You may need to install the SRS if not done yet: https://launchpad.support.sap.com/#/notes/2810660

In [ ]:
# %sql CREATE PREDEFINED SPATIAL REFERENCE SYSTEM IDENTIFIED BY 5018

# Prepare, Persists and Enhance Trajectory Data

## Prepare Data in Python

Download CSV from https://www.kaggle.com/crailtap/taxi-trajectory and reference the respective file below.

In [ ]:
%%time
df_csv = pd.read_csv('/Users/d059468/Downloads/train.csv')

When looking at the data, we can see that POLYLINE is not WKT and Timestamp is in UNIX format

In [ ]:
df_csv.head(5)

Convert the timestamp to proper datetime

In [ ]:
df_csv["TIMESTAMP"] = pd.to_datetime(df_csv['TIMESTAMP'],unit='s')

Build WKT. I am a regex and Python noob and just do it old school. I appreciate any hints for making this more elegant :)

In [ ]:
%%time
df_csv["POLYLINE"] = df_csv["POLYLINE"].str.replace(",", " ").str.replace("\] \[", ",").str.replace("\[\[", "LINESTRING(").str.replace("\]\]",")")
df_csv["POLYLINE"] = df_csv["POLYLINE"].str.replace("\[\]", "LINESTRING EMPTY")

There are some empty trajectories. These are not necessarily an issue.

In [ ]:
df_csv["POLYLINE"].str.contains('LINESTRING EMPTY').sum()

However, there is a large number of LINESTRINGS containing only one point. From a geometrical perspective this is not a valid linestring, which is why SAP HANA will not allow to create this objects with type linestring.

In [ ]:
df_csv["POLYLINE"].str.contains('LINESTRING\(-?\d*\.?\d*\s*-?\d*\.?\d*\)').sum()

We just filter these suspicious and empty trajectories as it does not add benefit to our analysis.

In [ ]:
%%time
df_csv = df_csv[~df_csv["POLYLINE"].str.contains('LINESTRING EMPTY')]
df_csv = df_csv[~df_csv["POLYLINE"].str.contains('LINESTRING\(-?\d*\.?\d*\s*-?\d*\.?\d*\)')]
df_csv.shape

Filter data based on date. This amount of data will fit into a HANA Express instance.

In [ ]:
print("Date range: %s - %s " % (df_csv["TIMESTAMP"].min(), df_csv["TIMESTAMP"].max()))

In [ ]:
df_csv = df_csv[(df_csv["TIMESTAMP"] >= '2013-12-01') & (df_csv["TIMESTAMP"] <= '2014-02-01')]
df_csv.shape

Polyline is now a proper WKT and Timestamp a readable date/time format

In [ ]:
df_csv.head(5)

## Persist Trajectory Data

Write the rest to the database

In [ ]:
%%time
hdb_connection = sqlalchemy.create_engine(connection_string).connect()

obj_cols = df_csv.select_dtypes(include=[object]).columns.values.tolist()
obj_cols.remove('POLYLINE')
df_csv.to_sql(name = 'taxi', schema=hdb_schema, con = hdb_connection, if_exists = 'replace', chunksize = 500, dtype={c: sqlalchemy.types.String(512) for c in obj_cols})

Add a proper spatial dimension

In [ ]:
%sql ALTER TABLE TAXI ADD (SHAPE ST_GEOMETRY($srid))
%sql UPDATE TAXI SET SHAPE = ST_GEOMFROMTEXT(POLYLINE, 4326).ST_TRANSFORM($srid)
%sql ALTER TABLE TAXI DROP (POLYLINE)

Verify that data has arrived

In [ ]:
%sql SELECT COUNT(*) FROM TAXI

## Excursus: Why not always use 3857 when going to planar projection?

Avg travel distance with round earth SRS

In [ ]:
# sql_result = %sql SELECT AVG(SHAPE.ST_TRANSFORM(4326).ST_LENGTH()) FROM TAXI
# dist_4326 = float(sql_result[0][0])
# dist_4326

Avg travel distance with webmercator projection

In [ ]:
# sql_result = %sql SELECT AVG(SHAPE.ST_TRANSFORM(3857).ST_LENGTH()) FROM TAXI
# dist_3857 = float(sql_result[0][0])
# dist_3857

Avg travel distance with projection suitable for Porto

In [ ]:
# sql_result = %sql SELECT AVG(SHAPE.ST_TRANSFORM(5018).ST_LENGTH()) FROM TAXI
# dist_5018 = float(sql_result[0][0])
# dist_5018

In [ ]:
# print('Deviation 3857: %f percent' % (100 * (dist_3857 / dist_4326 - 1)))
# print('Deviation 5018: %f percent' % (100 * (dist_5018 / dist_4326 - 1)))

## Enhance dataset

Add start and end point

In [ ]:
%sql ALTER TABLE TAXI ADD (STARTPOINT ST_GEOMETRY($srid), ENDPOINT ST_GEOMETRY($srid))
%sql UPDATE TAXI SET STARTPOINT = SHAPE.ST_STARTPOINT(), ENDPOINT = SHAPE.ST_ENDPOINT()

Add trip duration in seconds

In [ ]:
%sql ALTER TABLE TAXI ADD (DURATION INTEGER)
%sql UPDATE TAXI SET DURATION = (SHAPE.ST_NUMPOINTS() - 1) * 15

Add start and end time (start time just for the sake of consistent naming)

In [ ]:
%sql ALTER TABLE TAXI ADD (STARTTIME TIMESTAMP, ENDTIME TIMESTAMP)
%sql UPDATE TAXI SET STARTTIME = TIMESTAMP, ENDTIME = ADD_SECONDS(TIMESTAMP, DURATION)

Add driving distance in meter

In [ ]:
%sql ALTER TABLE TAXI ADD (DISTANCE INTEGER)
%sql UPDATE TAXI SET DISTANCE = TO_INTEGER(SHAPE.ST_LENGTH())

Add average speed in km/h

In [ ]:
%sql ALTER TABLE TAXI ADD (SPEED_AVG INTEGER)
%sql UPDATE TAXI SET SPEED_AVG = TO_INTEGER(DISTANCE/DURATION * 3.6)

Flag weekend days

In [ ]:
%sql ALTER TABLE TAXI ADD (IS_WEEKEND INTEGER)
%sql UPDATE TAXI SET IS_WEEKEND = CASE WHEN WEEKDAY(TIMESTAMP) >= 5 THEN 1 ELSE 0 END

Flag Holidays

In [ ]:
%sql ALTER TABLE TAXI ADD (IS_HOLIDAY INTEGER)
%sql UPDATE TAXI SET IS_HOLIDAY = CASE WHEN DAY_TYPE='B' THEN 1 ELSE 0 END

Flag days before holidays or weekends

In [ ]:
%sql ALTER TABLE TAXI ADD (IS_PARTYNIGHT INTEGER)
%sql UPDATE TAXI SET IS_PARTYNIGHT = CASE WHEN DAY_TYPE='C' OR WEEKDAY(TIMESTAMP) = 4 THEN 1 ELSE 0 END

Data prep finished. Perform delta merge.

In [ ]:
%sql MERGE DELTA OF TAXI

## Consistency Checks

Trips without actual trajectories

(We actually removed those beforehand)

In [ ]:
%sql SELECT COUNT(*) FROM TAXI WHERE SHAPE.ST_ISEMPTY() = 1

Missing GPS data

In [ ]:
%sql SELECT COUNT(*) FROM TAXI WHERE MISSING_DATA > 0

Invalid trajectories (in terms of geometry)

In [ ]:
%sql SELECT COUNT(*) FROM TAXI WHERE SHAPE.ST_ISVALID() = 0

Actually, all of those contain only two identical points

In [ ]:
%sql SELECT COUNT(*) FROM TAXI WHERE SHAPE.ST_ISVALID() = 0 AND STARTPOINT != ENDPOINT

(Too) short trips

In [ ]:
%sql SELECT COUNT(*) FROM TAXI WHERE DISTANCE < 200

(Too) fast trips 

(But what is too fast? https://www.youtube.com/watch?v=buQxF0eIeXI)

In [ ]:
%sql SELECT COUNT(*) FROM TAXI WHERE SPEED_AVG > 150

### Remove inconsistent data

In [ ]:
%sql DELETE FROM TAXI WHERE SHAPE.ST_ISEMPTY() = 1 OR SHAPE.ST_ISVALID() = 0 OR DISTANCE < 200 OR SPEED_AVG > 150 OR MISSING_DATA > 0

# Download and Persist POIs for Porto

Retrieve the polygon for which we would like to download POI data

In [ ]:
sql_result = %sql SELECT ST_CONVEXHULLAGGR(SHAPE).ST_TRANSFORM(4326).ST_ASWKT() FROM TAXI
df_poi_shape = sql_result.DataFrame()

Just check which area has been selected

In [ ]:
KeplerGl(height=500, data={'poi_shape':df_poi_shape})

Load POIs from relevant categories. (see https://wiki.openstreetmap.org/wiki/Key:amenity)

In [ ]:
%%time

# query has strong selectivity and should not be split up by area size
ox.config(max_query_area_size = 1000000000000)

gdf_poi = ox.geometries.geometries_from_polygon(
    df_poi_shape[df_poi_shape.columns[0]].apply(wkt.loads).iloc[0],
    tags = {"amenity":[
                'taxi', 'car_rental', 'bus_station',            #transportation
                'bar', 'restaurant', 'pub', 'cafe',             #sustenance
                'university', 'college',                        #education
                'clinic', 'doctors', 'hospital', 'pharmacy'     #healthcare
                'cinema', 'nightclub', 'stripclub', 'theater',  #entertainment
                'conference_centre'
    ]}
)
gdf_poi.shape

Convert to dataframe and handle datatypes

In [ ]:
df_poi = pd.DataFrame(gdf_poi)
df_poi = df_poi.reset_index()
df_poi = df_poi[['osmid', 'geometry', 'amenity', 'name']]
df_poi["geometry"] = df_poi["geometry"].astype("str")
df_poi = df_poi.infer_objects()

Persist in database

In [ ]:
%%time
hdb_connection = sqlalchemy.create_engine(connection_string).connect()
obj_cols = df_poi.select_dtypes(include=[object]).columns.values.tolist()
obj_cols.remove('geometry')
df_poi.to_sql(name = 'osm_poi', schema=hdb_schema, con = hdb_connection, if_exists = 'replace', chunksize = 100, dtype={c:sqlalchemy.types.String(512) for c in obj_cols})

Add ST_GEOMETRY column

In [ ]:
%sql ALTER TABLE OSM_POI ADD (SHAPE ST_GEOMETRY($srid))
%sql UPDATE OSM_POI SET SHAPE = ST_GEOMFROMTEXT(GEOMETRY, 4326).ST_TRANSFORM($srid)
%sql ALTER TABLE OSM_POI DROP (GEOMETRY)

Verify that data has arrived

In [ ]:
%sql SELECT COUNT(*) FROM OSM_POI

# Add Reference Hexagonal Grid

For further analysis it will be helpful to persist a hexagonal grid covering all relevant points

In [ ]:
%sql DROP TABLE REFGRID
%sql CREATE COLUMN TABLE REFGRID (HEXID VARCHAR(50), HEXCELL ST_GEOMETRY($srid), HEXCENTROID ST_GEOMETRY($srid))

In [ ]:
%%sql
INSERT INTO REFGRID
(
    SELECT 
        'HEXID-'|| ST_CLUSTERID() AS HEXID,
        ST_CLUSTERCELL() AS HEXCELL, 
        ST_CLUSTERCELL().ST_CENTROID() AS HEXCENTROID
    FROM ((SELECT STARTPOINT AS PT FROM TAXI) UNION (SELECT ENDPOINT AS PT FROM TAXI))
    GROUP CLUSTER BY PT USING HEXAGON X CELLS 250
)

Take a quick look at the refgrid (..also including the convex hull of all points)

In [ ]:
%%sql sql_result <<
SELECT 
    HEXID, 
    HEXCENTROID.ST_TRANSFORM(4326).ST_ASWKT() AS HEXCENTROID,
    HEXCELL.ST_TRANSFORM(4326).ST_ASWKT() AS HEXCELL
FROM REFGRID

In [ ]:
df_refgrid = sql_result.DataFrame()

In [ ]:
KeplerGl(height=500, data={'refgrid':df_refgrid, 'poi_shape':df_poi_shape})

# Some Basic Spatial Analytics

## The very basics

Average ride distance

In [ ]:
%sql SELECT AVG(DISTANCE)/1000 AS DISTANCE_KM FROM TAXI

Average ride duration

In [ ]:
%sql SELECT AVG(DURATION)/60 AS DURATION_MINUTES FROM TAXI

Average speed

In [ ]:
%sql SELECT AVG(SPEED_AVG) AS SPEED_KMH FROM TAXI

## Visualize samples using kepler.gl

Fetch some sample trajectories

In [ ]:
%%sql sql_result << 
SELECT TOP 1000
    INDEX, 
    TRIP_ID, 
    CALL_TYPE, 
    TAXI_ID, 
    STARTTIME, 
    ENDTIME,
    SPEED_AVG,
    SHAPE.ST_TRANSFORM(4326).ST_ASWKT() as SHAPE
FROM TAXI
ORDER BY RAND()

In [ ]:
df_sample_trajectories = sql_result.DataFrame()

Configure and display map

In [ ]:
map_sample_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 41.16064263660347,
            'longitude': -8.61937846161915,
            'zoom': 10.936755405111594
        }
    }
}

KeplerGl(height=500, data={'samples':df_sample_trajectories}, config=map_sample_config)

Also get the POIs on the map

In [ ]:
sql_result = %sql SELECT OSMID, SHAPE.ST_TRANSFORM(4326).ST_ASWKT() AS SHAPE, AMENITY, NAME FROM OSM_POI
df_all_poi = sql_result.DataFrame()

In [ ]:
KeplerGl(height=500, data={'pois':df_all_poi, 'samples':df_sample_trajectories}, config=map_sample_config)

# Analyze Pick-up Locations

In [ ]:
%%sql sql_result <<
SELECT 
    ST_CLUSTERID(),
    ST_CLUSTERCELL().ST_TRANSFORM(4326).ST_ASGEOJSON() AS HEXCELL,
    LOG(10, COUNT(*)) AS QUANTITY
FROM TAXI
GROUP CLUSTER BY STARTPOINT USING HEXAGON X CELLS 500

In [ ]:
df_pickup_hex = sql_result.DataFrame()

Configure map visualization

In [ ]:
map_pickup_hex_config = {'version': 'v1',
 'config': {'visState': {
   'layers': [{
     'id': 'k6a7rbn',
     'type': 'geojson',
     'config': {
      'dataId': 'hex',
      'label': 'hex',
      'color': [241, 92, 23],
      'columns': {'geojson': 'hexcell'},
      'isVisible': True,
      'visConfig': {
       'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [34, 63, 154],
       'colorRange': {'name': 'Uber Viz Diverging 1.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C',
         '#5DBABF',
         '#BAE1E2',
         '#F8C0AA',
         '#DD7755',
         '#C22E00']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': False,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'quantity', 'type': 'real'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': {'name': 'quantity', 'type': 'real'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {
   'bearing': 115.5596330275229,
   'dragRotate': True,
   'latitude': 41.191169915709146,
   'longitude': -8.631325549115484,
   'pitch': 57.45876401383432,
   'zoom': 11.013713014514414,
   'isSplit': False},
  }}

Visualize map

In [ ]:
KeplerGl(height=500, data={'hex':df_pickup_hex}, config=map_pickup_hex_config)

### POIs in the Cluster Cells with Most Pick-ups

In [ ]:
%%sql sql_result <<
SELECT B.OSMID, B.SHAPE.ST_TRANSFORM(4326).ST_ASWKT() AS OSMSHAPE, B.AMENITY, B.NAME, A.HEXCELL.ST_TRANSFORM(4326).ST_ASWKT() AS HEXSHAPE
FROM 
(
    SELECT TOP 3 ST_CLUSTERCELL() AS HEXCELL
    FROM TAXI
    GROUP CLUSTER BY STARTPOINT USING HEXAGON X CELLS 500
    ORDER BY COUNT(*) DESC
) A LEFT JOIN OSM_POI B ON A.HEXCELL.ST_INTERSECTS(B.SHAPE) = 1

In [ ]:
df_top_cells = sql_result.DataFrame()

In [ ]:
map_top_cells_config = {
    'version': 'v1',
    'config': {
        'mapState': {
           'latitude': 41.14581779896211,
           'longitude': -8.598703907021486,
           'zoom': 13.933597056454914
        }
    }
}

KeplerGl(height=500, data={'top_cells':df_top_cells}, config=map_top_cells_config)

## Pick-up Locations Over Time

Fetch hex clusters over time. Only pick the ones having more than 1 occurence to make the visualization less noisy.

In [ ]:
%%sql sql_result <<
SELECT 
    CLUSTERID, 
    CLUSTERCELL.ST_TRANSFORM(4326).ST_ASGEOJSON() AS CLUSTERCELL, 
    HOURBIN,
    LOG(10, COUNT(*)) AS QUANTITY
FROM
(
    SELECT 
        TO_TIMESTAMP(YEAR(STARTTIME) || '-' || MONTH(STARTTIME) || '-' || DAYOFMONTH(STARTTIME) || ' ' || LPAD(HOUR(STARTTIME) - MOD(HOUR(STARTTIME),2), 2, '0') || ':00:00') AS HOURBIN,
        ST_CLUSTERID() OVER (CLUSTER BY STARTPOINT USING HEXAGON X CELLS 250) AS CLUSTERID,
        ST_CLUSTERCELL() OVER (CLUSTER BY STARTPOINT USING HEXAGON X CELLS 250) AS CLUSTERCELL,
        TRIP_ID
    FROM TAXI
)
GROUP BY CLUSTERID, CLUSTERCELL, HOURBIN
HAVING COUNT(*) > 1

In [ ]:
df_pickup_time = sql_result.DataFrame()

In [ ]:
map_pickup_time_config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': 'timebins',
     'id': '2clyivov',
     'name': 'hourbin',
     'type': 'timeRange',
     'value': [1385856000000, 1385863200000],
     'enlarged': True,
     'plotType': 'histogram',
     'yAxis': None}],
   'layers': [{'id': 'nngj5g8j',
     'type': 'geojson',
     'config': {'dataId': 'timebins',
      'label': 'timebins',
      'color': [248, 149, 112],
      'columns': {'geojson': 'clustercell'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [130, 154, 227],
       'colorRange': {'name': 'Uber Viz Diverging 1.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C',
         '#5DBABF',
         '#BAE1E2',
         '#F8C0AA',
         '#DD7755',
         '#C22E00']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'quantity', 'type': 'real'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': {'name': 'quantity', 'type': 'real'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'timebins': ['clusterid',
       'hourbin',
       'numberbin',
       'quantity']},
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 112.9908256880734,
   'dragRotate': True,
   'latitude': 41.20398275560239,
   'longitude': -8.67967113104948,
   'pitch': 52.77444039813042,
   'zoom': 10.424667679276855,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [ ]:
KeplerGl(height=700, data={'timebins':df_pickup_time}, config=map_pickup_time_config)

# The Route to the Airport

## Relation between Start and Destination

Query and visualize all start/end combinations, which have been observed with more than 100 trips.

In [ ]:
%%sql sql_result <<
SELECT 
        START_HEXID,
        START_CENTROID.ST_TRANSFORM(4326).ST_X() AS START_CELL_LON,
        START_CENTROID.ST_TRANSFORM(4326).ST_Y() AS START_CELL_LAT,
        END_HEXID,
        END_CENTROID.ST_TRANSFORM(4326).ST_X() AS END_CELL_LON,
        END_CENTROID.ST_TRANSFORM(4326).ST_Y() AS END_CELL_LAT,
        COUNT(*) AS CNT
FROM 
(
    SELECT 
        TRIP_ID, 
        a.HEXID AS START_HEXID,
        a.HEXCENTROID AS START_CENTROID,
        b.HEXID AS END_HEXID,
        b.HEXCENTROID AS END_CENTROID
    FROM TAXI
    LEFT JOIN REFGRID a ON STARTPOINT.ST_WITHIN(a.HEXCELL) = 1
    LEFT JOIN REFGRID b ON ENDPOINT.ST_WITHIN(b.HEXCELL) = 1
)
GROUP BY START_HEXID, START_CENTROID, END_HEXID, END_CENTROID
HAVING COUNT(*) > 100

In [ ]:
df_cell_relation = sql_result.DataFrame()

In [ ]:
config_cell_relation = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'j9i3lca',
     'type': 'arc',
     'config': {'dataId': 'cell relation',
      'label': 'cell relation',
      'color': [207, 237, 181],
      'columns': {'lat0': 'start_cell_lat',
       'lng0': 'start_cell_lon',
       'lat1': 'end_cell_lat',
       'lng1': 'end_cell_lon'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 2,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [245, 153, 153]},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': {'name': 'cnt', 'type': 'integer'},
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'cell relation': ['start_hexid',
       'cnt',
       'end_hexid']},
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 26.752293577981668,
   'dragRotate': True,
   'latitude': 41.1926903030073,
   'longitude': -8.61496918743284,
   'pitch': 57.99119946737215,
   'zoom': 11.287843857109973,
   'isSplit': False},
  'mapStyle': {'styleType': 'light',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

KeplerGl(height=500, data={'cell relation':df_cell_relation}, config=config_cell_relation)

With the visualization above we can anticipate that most trips to the airport start in the area around Sao Bento station.

### What Is The Best Way From Sao Bento Station To The Airport?

Query and visualize all trips from Sao Bento Station to the airport

In [ ]:
%%sql sql_result <<
SELECT
    INDEX, 
    TRIP_ID, 
    CALL_TYPE, 
    TAXI_ID, 
    STARTTIME, 
    ENDTIME,
    SPEED_AVG,
    DURATION,
    DISTANCE,
    SHAPE.ST_TRANSFORM(4326).ST_ASWKT() as SHAPE,
    a.HEXCELL.ST_TRANSFORM(4326).ST_ASWKT() AS START_HEXCELL,
    b.HEXCELL.ST_TRANSFORM(4326).ST_ASWKT() AS END_HEXCELL
FROM TAXI t
LEFT JOIN REFGRID a ON STARTPOINT.ST_WITHIN(a.HEXCELL) = 1
LEFT JOIN REFGRID b ON ENDPOINT.ST_WITHIN(b.HEXCELL) = 1
WHERE a.HEXID = 'HEXID-86826' AND b.HEXID = 'HEXID-90071' AND DISTANCE < 2 * a.HEXCENTROID.ST_DISTANCE(b.HEXCENTROID)

In [ ]:
df_frequent_route = sql_result.DataFrame()

In [ ]:
config_frequent_route = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'byb7s0c',
     'type': 'geojson',
     'config': {'dataId': 'frequent route',
      'label': 'frequent route',
      'color': [130, 154, 227],
      'columns': {'geojson': 'shape'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Uber Viz Diverging 1.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C',
         '#5DBABF',
         '#BAE1E2',
         '#F8C0AA',
         '#DD7755',
         '#C22E00']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': False,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': {'name': 'duration', 'type': 'integer'},
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': '9phu6iq',
     'type': 'geojson',
     'config': {'dataId': 'frequent route',
      'label': 'frequent route',
      'color': [231, 159, 213],
      'columns': {'geojson': 'start_hexcell'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [30, 150, 190],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'e7hwsd',
     'type': 'geojson',
     'config': {'dataId': 'frequent route',
      'label': 'frequent route',
      'color': [137, 218, 193],
      'columns': {'geojson': 'end_hexcell'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [179, 173, 158],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'frequent route': ['index',
       'trip_id',
       'call_type',
       'taxi_id',
       'starttime']},
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 41.190118850547385,
   'longitude': -8.63244718721786,
   'pitch': 0,
   'zoom': 11.044961898535204,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

KeplerGl(height=500, data={'frequent route':df_frequent_route}, config=config_frequent_route)

# HANA Embedded Machine Learning

Now we will use the hana_ml client for db communication. This way we can make sure, that the data resides in the database and gets processed by embedded ML. I.e. HANA ML DataFrame object will not have a persistence in Python unless 'collect()' gets called.

In [ ]:
from hana_ml import dataframe
from hana_ml.algorithms.apl import regression
from matplotlib import pyplot
from hana_ml.algorithms.apl import gradient_boosting_classification

Re-usable function for retrieving performance metrics as dataframe

In [ ]:
def performance_metrics_df(model):
    d = model.get_performance_metrics()
    df = pd.DataFrame(list(d.items()), columns=["Metric", "Value"])
    return df

Re-usable function for plotting feature importance

In [ ]:
def plot_feature_importance(model):
    # retrieve importance as df
    d = model.get_feature_importances()
    df = pd.DataFrame(list(d.items()), columns=["Variable", "Contribution"])
    df['Contribution'] = df['Contribution'].astype(float)
    df['Cumulative'] = df['Contribution'].cumsum()
    df['Contribution'] = df['Contribution'].round(4)*100
    df['Cumulative'] = df['Cumulative'].round(4)*100
    non_zero = df['Contribution'] != 0
    dfs = df[non_zero].sort_values(by=['Contribution'], ascending=False)
    
    # visualize importance as bar chart
    c_title = "Contributions"
    dfs = dfs.sort_values(by=['Contribution'], ascending=True)
    dfs.plot(kind='barh', x='Variable', y='Contribution', title=c_title,legend=False, fontsize=12)
    pyplot.show()

Re-usable function for plotting the group significance of a feature

In [ ]:
def plot_group_significance(model, feature):
    df = model.get_indicators().filter("VARIABLE='" + feature + "' and KEY='GroupSignificance'").collect()
    df['VALUE'] = df['VALUE'].astype(float)
    df.sort_values('VALUE', inplace = True, ascending = False)
    
    c_title = "Significance"
    df.plot(kind='barh', x='DETAIL', y='VALUE', title=c_title,legend=False, fontsize=12)
    pyplot.show()

Establish connection

In [ ]:
conn = dataframe.ConnectionContext(hdb_host, hdb_port, hdb_user, hdb_password)
conn.sql('SET SCHEMA %s' % (hdb_schema))

## Predict the duration of a trip

In [ ]:
hdf_trajectories = conn.sql('''
    SELECT
        INDEX,
        STARTTIME,
        R1.HEXID AS HEXID_START,
        R2.HEXID AS HEXID_END,
        DURATION
    FROM TAXI
    LEFT JOIN REFGRID R1 ON STARTPOINT.ST_WITHIN(R1.HEXCELL) = 1
    LEFT JOIN REFGRID R2 ON ENDPOINT.ST_WITHIN(R2.HEXCELL) = 1
''')

Instanciate regression model

In [ ]:
regr_model = regression.AutoRegressor(conn_context = conn, variable_auto_selection = True)

Train model or alternatively load model from HANA

In [ ]:
%%time
# Train model:
regr_model.fit(hdf_trajectories, label='DURATION', features=['STARTTIME', 'HEXID_START', 'HEXID_END'], key='INDEX')

# Load pre-trained model:
# regr_model.load_model(hdb_schema, 'MODEL_DURATION')
# regr_model.indicators_ = conn.sql('SELECT * FROM MODEL_DURATION_INDICATORS')

Save model in HANA

In [ ]:
regr_model.save_artifact(regr_model.indicators_, hdb_schema, 'MODEL_DURATION_INDICATORS', if_exists='replace')
regr_model.save_model(hdb_schema, 'MODEL_DURATION', if_exists='replace')

Evaluate the model performance

In [ ]:
performance_metrics_df(regr_model)

Analyze the variable importance

In [ ]:
plot_feature_importance(regr_model)

Analyze the significance of a certain dimension (e.g. Hour of day)

In [ ]:
plot_group_significance(regr_model, 'STARTTIME_H')

Make a prediction for the trip to the airport

In [ ]:
hdf_predict = conn.sql('''
    SELECT
        0 INDEX,
        '2020-02-10 20:00:00' as STARTTIME,
        'HEXID-86826' AS HEXID_START,
        'HEXID-90071' AS HEXID_END
    FROM DUMMY
''')

In [ ]:
regr_model.predict(hdf_predict).collect()

Check the feasibility yourself: https://www.google.com/maps/dir/S%C3%A3o+Bento+Station,+Pra%C3%A7a+de+Almeida+Garrett,+Porto,+Portugal/Francisco+S%C3%A1+Carneiro+Airport+(OPO),+Maia,+Portugal/

## Predict where a taxi ride is going to end

There is a limitation of APL: A maximum of 100 target classes is allowed. Let's see what the influence on our analysis will be when only considering the 100 most frequent destinations in Porto.

In [ ]:
%%sql
SELECT SUM(NTRIPS) AS TOP100_LOC, 100 * SUM(NTRIPS) / (SELECT COUNT(*) FROM TAXI) AS PERCENTAGE
FROM
(
    SELECT TOP 100 HEXID, COUNT(*) as NTRIPS
    FROM TAXI
    LEFT JOIN REFGRID ON ENDPOINT.ST_WITHIN(HEXCELL) = 1
    GROUP BY HEXID
    ORDER BY COUNT(*) DESC
)

This means, when only looking at the top 100 taxi destinations we actually cover almost all trips. I would argue that the remaining destinations actually have a so little number of trips, that the prediction would be unstable anyway.

Add a compass to see in which direction the taxi was going after the first 5 coordinates

In [ ]:
%sql ALTER TABLE TAXI ADD (COMPASS NVARCHAR(2))
%sql ALTER TABLE TAXI ADD (COMPASS_DIST INTEGER)

In [ ]:
%%sql 
UPDATE TAXI T SET 
    COMPASS = 
        CASE 
            WHEN 1 - ABS(D.DIR) <= 0.125 THEN 'W'
            WHEN ABS(0 - D.DIR) <= 0.125 THEN 'E'
            WHEN ABS(0.5 - D.DIR) <= 0.125 THEN 'N'
            WHEN ABS(-0.5 - D.DIR) <= 0.125 THEN 'S'
            WHEN ABS(0.25 - D.DIR) < 0.125 THEN 'NE'
            WHEN ABS(0.75 - D.DIR) < 0.125 THEN 'NW'
            WHEN ABS(-0.25 - D.DIR) < 0.125 THEN 'SE'
            WHEN ABS(-0.75 - D.DIR) < 0.125 THEN 'SW'
            ELSE NULL
        END,
    COMPASS_DIST = STARTPOINT.ST_DISTANCE(SHAPE.ST_POINTN(10))
FROM
    TAXI T,
    (
        SELECT TRIP_ID, 0.5 * atan2(SHAPE.ST_POINTN(10).ST_Y() - STARTPOINT.ST_Y(), SHAPE.ST_POINTN(10).ST_X() - STARTPOINT.ST_X())/acos(0) AS DIR
        FROM TAXI
        WHERE SHAPE.ST_NUMPOINTS() > 10
    ) D
WHERE T.TRIP_ID = D.TRIP_ID

Number of sample records to fetch

In [ ]:
n_samples = 75000

Gather training data and consider only rides having enought trajectory points

In [ ]:
hdf_rides = conn.sql('''
    SELECT *, RANDOM_PARTITION(0.8, 0.0, 0.2, 0) OVER (ORDER BY STARTTIME) AS SET_NUM
    FROM
    (
        SELECT TOP %s
            TRIP_ID,
            CALL_TYPE,
            DAY_TYPE,
            STARTTIME,
            COMPASS,
            COMPASS_DIST,
            a.HEXID AS START_HEXID,
            b.HEXID AS END_HEXID
        FROM TAXI
        LEFT JOIN REFGRID a ON STARTPOINT.ST_WITHIN(a.HEXCELL) = 1
        LEFT JOIN REFGRID b ON ENDPOINT.ST_WITHIN(b.HEXCELL) = 1
        WHERE 
            COMPASS IS NOT NULL
        AND b.HEXID IN
        (
            SELECT TOP 100 HEXID
            FROM TAXI
            LEFT JOIN REFGRID ON ENDPOINT.ST_WITHIN(HEXCELL) = 1
            GROUP BY HEXID
            ORDER BY COUNT(*) DESC
        )
        ORDER BY RAND()
    )
''' % (n_samples))

In [ ]:
hdf_rides.head(5).collect()

In [ ]:
# hdf_rides.count()

In [ ]:
# hdf_rides.filter('SET_NUM=1').count() #training

In [ ]:
# hdf_rides.filter('SET_NUM=3').count() #test

Instantiate multi-classification model

In [ ]:
gb_model = gradient_boosting_classification.GradientBoostingClassifier(conn)

Train model or alternatively load model from HANA

In [ ]:
%%time
# Train model
gb_model.fit(
    hdf_rides.filter('SET_NUM=1'), 
    label='END_HEXID', 
    key = 'TRIP_ID',
    features = ['CALL_TYPE', 'DAY_TYPE', 'STARTTIME', 'COMPASS', 'COMPASS_DIST', 'START_HEXID'])

# Load pre-trained model:
# gb_model.load_model(hdb_schema, 'MODEL_DESTINATION')
# gb_model.indicators_ = conn.sql('SELECT * FROM MODEL_DESTINATION_INDICATORS')
# gb_model.summary_ = conn.sql('SELECT * FROM MODEL_DESTINATION_SUMMARY')
# gb_model.label = 'END_HEXID'

Save model to HANA

In [ ]:
gb_model.save_artifact(gb_model.indicators_, hdb_schema, 'MODEL_DESTINATION_INDICATORS', if_exists='replace')
gb_model.save_artifact(gb_model.summary_, hdb_schema, 'MODEL_DESTINATION_SUMMARY', if_exists='replace')
gb_model.save_model(hdb_schema, 'MODEL_DESTINATION', if_exists='replace')

In [ ]:
performance_metrics_df(gb_model)

In [ ]:
gb_model.get_feature_importances()

### Predictions on Test Data

Do some predictions on the test dataset

In [ ]:
%%time
hdf_predict = gb_model.predict(hdf_rides.filter('SET_NUM=3'))

Join the geo information back

In [ ]:
hdf_predict_refgrid = hdf_predict.join(
        conn.table('REFGRID'), 
        'TRUE_LABEL = HEXID', 
        select=[('TRIP_ID'), ('TRUE_LABEL'), ('PREDICTED'), ('PROBABILITY'), ('HEXCENTROID', 'TRUE_CENTROID')]
    )
hdf_predict_refgrid = hdf_predict_refgrid.join(
        conn.table('REFGRID'), 
        'PREDICTED = HEXID', 
        select=[('TRIP_ID'), ('TRUE_LABEL'), ('PREDICTED'), ('PROBABILITY'), ('TRUE_CENTROID'), ('HEXCENTROID', 'PREDICTED_CENTROID')]
    )

In [ ]:
hdf_predict_refgrid.head(5).collect()

Calculate distance between predicted and true centroids

In [ ]:
hdf_predict_refgrid = hdf_predict_refgrid.select('*', ('TRUE_CENTROID.ST_DISTANCE(PREDICTED_CENTROID)', 'DIST'))

Calculate median distance of our predictions as a quality indicator

In [ ]:
hdf_predict_refgrid.agg([('AVG', 'DIST', 'AVG_DIST')]).collect()

To set this into context, we need to know the size of our hexagons

In [ ]:
%%sql
SELECT TOP 1 
    HEXCELL.ST_EXTERIORRING().ST_POINTN(1).ST_DISTANCE(HEXCELL.ST_EXTERIORRING().ST_POINTN(4)) AS HEX_DIAMETER
FROM REFGRID

In [ ]:
tp_prediction = hdf_predict_refgrid.filter('DIST < 1100').count()
tp_prediction

(Nearly) correct predictions

In [ ]:
print('%s%%' % (100 * tp_prediction / hdf_predict_refgrid.count()))

### Benchmark with a majority vote

Which cell is the most frequent destination?

In [ ]:
hdf_top_destination = conn.sql('''
    SELECT TOP 1 HEXID, HEXCENTROID
    FROM TAXI
    LEFT JOIN REFGRID ON ENDPOINT.ST_WITHIN(HEXCELL) = 1
    GROUP BY HEXID, HEXCENTROID
    ORDER BY COUNT(*) DESC
''')

Add frequent vote to prediction df

In [ ]:
hdf_predict_frequent = hdf_predict_refgrid.join(
        hdf_top_destination, 
        "1=1", 
        select=[('TRIP_ID'), ('TRUE_LABEL'), ('PREDICTED'), ('PROBABILITY'), ('TRUE_CENTROID'), ('PREDICTED_CENTROID'), ('HEXCENTROID', 'FREQUENT_CENTROID')]
    ).select(
        '*', ('TRUE_CENTROID.ST_DISTANCE(FREQUENT_CENTROID)', 'FREQUENT_DIST')
    )

In [ ]:
hdf_predict_frequent.agg([('AVG', 'FREQUENT_DIST', 'AVG_FREQUENT_DIST')]).collect()

In [ ]:
tp_majority = hdf_predict_frequent.filter('FREQUENT_DIST < 1100').count()
tp_majority

Increase in classification rate with model compared to majority vote

In [ ]:
print('%s%%' % (100 * (tp_prediction - tp_majority) / hdf_predict_refgrid.count()))

Fin.